In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"

pc = pd.read_csv('parking_space_count.csv')

In [3]:
pc

,Unnamed: 0,block_id,side,type,length,cnt
0,0,01-24,NE,CVLZ,23,1
1,1,01-24,NE,PLZ,22,1
2,2,01-24,NE,PS,143,8
3,3,01-24,SW,PLZ,38,2
4,4,01-24,SW,PS,58,3
...,...,...,...,...,...,...
75,75,27-02,SE,PS,88,5
76,76,27-03,NW,CVLZ,18,1
77,77,27-03,NW,PLZ,18,1
78,78,27-03,NW,PS,119,7


In [4]:
with open(add_xml, "w") as add:
    print("""<?xml version="1.0" encoding="UTF-8"?>

<additional xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/additional_file.xsd">

    <location netOffset="-362563.54,-4832311.19" convBoundary="0.00,0.00,2064.54,1284.45" origBoundary="13.296064,43.631354,13.321826,43.642858" projParameter="+proj=utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"/>
    """, file=add)

In [5]:
def side_2_roadway(block_id, side, edge_oneway_dict):
    if not edge_oneway_dict[block_id]:
        if side == 'SE':
            return block_id + '-' + 'NE'
        elif side == 'NE':
            return block_id + '-' + 'NW'
        elif side == 'NW':
            return block_id + '-' + 'SW'
        elif side == 'SW':
            return block_id + '-' + 'SE'
    else: 
        return edge_oneway_dict[block_id]

In [6]:
edges = []

root = ET.parse(net_xml).getroot()
for child in root.iter('edge'):
    if child.get('function') != 'internal':
        edges.append(child.get('id'))
        
edge_oneway_dict = {}
for edge_id in edges:
    if edge_id not in edge_oneway_dict:
        # if not checked, go ahead to check oneway
        if sum(word.startswith(edge_id[:5]) for word in edges) == 2:
            edge_oneway_dict[edge_id[:5]] = 0
        else:
            assert sum(word.startswith(edge_id[:5]) for word in edges) == 1
            edge_oneway_dict[edge_id[:5]] = edge_id

In [7]:
oneway_list = ['24-01', '24-02', '24-03', '25-02', '25-03', '26-01', '26-02', '26-03', '02-24', '02-25', '02-26', '04-24', '04-25', '04-26']
oneway_side_list = ['SE', 'SE', 'SE', 'NW', 'NW', 'SE', 'SE', 'SE', 'SW', 'SW', 'SW', 'NE', 'NE', 'NE']

In [8]:
oneway_side_dict = dict(zip(oneway_list, oneway_side_list))

In [9]:
pc_oneway = pc[pc['block_id'].isin(oneway_list)].groupby(['block_id','type']).sum().reset_index()
pc_oneway.drop(columns = ['Unnamed: 0'], inplace = True)
pc_oneway['side'] =  pc_oneway['block_id'].apply(lambda x: oneway_side_dict[x])
pc_oneway.shape

(26, 5)

In [10]:
pc_twoway = pc[~pc['block_id'].isin(oneway_list)]
pc_twoway.drop(columns = ['Unnamed: 0'], inplace = True)
pc_twoway.shape

C:\Users\Zhiyong\Anaconda3\envs\nmarl\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(38, 5)

### merge curbsides of oneway roads

In [11]:
pc_new = pd.concat([pc_oneway, pc_twoway])
pc_new.shape

(64, 5)

In [12]:
pc.shape

(80, 6)

In [13]:
pc_new

,block_id,type,length,cnt,side
0,02-24,CVLZ-B,46,1,SW
1,02-24,PS,196,10,SW
2,02-25,PLZ,19,1,SW
3,02-25,PS,343,18,SW
4,02-26,PLZ,75,4,SW
...,...,...,...,...,...
75,27-02,PS,88,5,SE
76,27-03,CVLZ,18,1,NW
77,27-03,PLZ,18,1,NW
78,27-03,PS,119,7,NW


### add one curb to 03-26-SW

In [14]:
pc_new['cnt'][(pc_new['block_id'] == '03-26') & (pc_new['side'] == 'SW') & (pc_new['type']=='PS')] = 2

C:\Users\Zhiyong\Anaconda3\envs\nmarl\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
pc_new[(pc_new['type']=='PS')]

,block_id,type,length,cnt,side
1,02-24,PS,196,10,SW
3,02-25,PS,343,18,SW
5,02-26,PS,202,11,SW
7,04-24,PS,242,13,NE
10,04-25,PS,280,15,NE
12,04-26,PS,275,16,NE
14,24-01,PS,313,25,SE
15,24-02,PS,287,16,SE
17,24-03,PS,55,3,SE
19,25-02,PS,216,12,NW


In [16]:
for index, row in pc_new.iterrows():
    block_id = row['block_id']
    side = row['side']
    space_type = row['type']
    length = row['length']
    count = row['cnt']
    lane = side_2_roadway(block_id, side, edge_oneway_dict) + '_0'
    parking_area_id = str(block_id) + '-' + str(side)
    with open(add_xml, "a") as add:
        if space_type == 'PS':
            print('    <parkingArea id="{}" lane="{}" startPos="10" endPos="50" roadsideCapacity="{}" angle="0" />'.\
#                   format(parking_area_id, lane, int(length)//16), # unit of curb length?
                  format(parking_area_id, lane, count), # unit of curb length?
                  file=add)

In [17]:
with open(add_xml, "a") as add:
    print('\n</additional>', file = add)